In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

import optuna

from common import EP
from dfdb import DFDB

import types
import copy

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [4]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [5]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [6]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [7]:
common_columns50 = ["number_peaks{'n': 5}",
 'abs_q75_6',
 'q01_roll_std_100',
 '5000crest_factor_quantile75',
 'abs_q01_4',
 'q25_roll_std_100',
 'q05_roll_std_10',
 'median__roll_std',
 'abs_q05_6',
 '5000form_factor_quantile75',
 '5000smoothness_quantile05',
 "quantile{'q': 0.9}",
 'abs_q75_2',
 'q01_roll_std_1000',
 '5000quantile75mean_',
 'spkt_welch_density__coeff_3',
 "number_peaks{'n': 10}",
 "number_peaks{'n': 1}",
 '5000smoothness_mean_',
 'abs_q25_5',
 '5000std_quantile05',
 '5000smoothness_std_',
 '5000smoothness_median_',
 '5000median_variance_',
 'spkt_welch_density__coeff_4',
 '5000variance_quantile25',
 'abs_q95_2',
 'abs_q75_7',
 'q05_5',
 '5000smoothness_entropy_',
 '5000smoothness_quantile25',
 'q01_2',
 'q05_roll_std_100',
 '3th_peak_freq',
 "autocorrelation{'lag': 5}",
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'q01_roll_std_10',
 'abs_q01_3',
 '5000variance_median_',
 'q75_roll_std_10',
 '5000skewness_max_',
 'iqr_3',
 'abs_q01_5',
 'q75_roll_mean_10',
 '5000quantile99median_',
 'iqr_6',
 'Hilbert_mean_6',
 'q05_roll_std_1000',
 "number_peaks{'n': 3}",
 'spkt_welch_densitycoeff_2',
 '5000std_median_',
 '5000std_quantile25']

In [8]:
common_columns75 = ['abs_q75_6',
 'abs_q01_4',
 'q25_roll_std_100',
 'q05_roll_std_10',
 'median__roll_std',
 '5000smoothness_quantile05',
 'spkt_welch_density__coeff_3',
 "number_peaks{'n': 10}",
 'abs_q25_5',
 'abs_q75_7',
 'q05_5',
 'q05_roll_std_100',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'iqr_6',
 'q05_roll_std_1000']

In [9]:
common_columns95 = ['abs_q75_6',
 'abs_q01_4',
 'q25_roll_std_100',
 "number_peaks{'n': 10}",
 'abs_q25_5',
 'q05_roll_std_100',
 'iqr_6',
 'q05_roll_std_1000']

In [10]:
db = DFDB('../trial2/svr.pkl', auto_commit=False)

In [20]:
param = {'algorithm': {'cls': 'SVR',
  'fit': {},
  'init': {'kernel': 'rbf',
   'degree': 2,
   'gamma': 'auto',
   'coef0': 0.0,
   'tol': 0.001,
   'C': 0.01,
   'epsilon': 0.01,
   'shrinking': True}},
 'columns': common_columns95,

 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},#stratified
 'scaler': {'cls': 'StandardScaler'}}

In [21]:
# run one try
mytrial = []
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='try common_columns95')
db.insert(mytrial[0])
df_trial = db.select()

In [22]:
df_trial[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-20 01:31:12.243686,52,2.032887,0.001813,2.134260,0.005904,0.101373,try common_columns50
1,2019-05-20 01:36:01.459672,15,2.047455,0.003158,2.092718,0.014978,0.045264,try common_columns75
2,2019-05-20 01:39:53.913244,8,2.053058,0.003195,2.095292,0.015183,0.042234,try common_columns95


In [23]:
db.commit()

In [ ]:
mytrial = []
#  tune hypterparameters
def objective(trial):
        
    tol = trial.suggest_uniform('tol', 0.0001, 0.01)
    C = trial.suggest_uniform('C', 0.0001, 0.1)
        
    args={'algorithm': {'cls': 'SVR',
      'fit': {},
      'init': {'kernel': 'rbf',
       'degree': 2,
       'gamma': 'auto',
       'coef0': 0.0,
       'tol': tol,
       'C': C,
       'epsilon': 0.01,
       'shrinking': True}},
     'columns': common_columns75,

     'kfold': {'n_splits': 3,
      'random_state': 1985,
      'shuffle': True,
      'type': 'group'},#stratified
     'scaler': {'cls': 'StandardScaler'}}
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-20 01:47:52,017] Finished a trial resulted in value: 0.2176318459171829. Current best value is 0.2176318459171829 with parameters: {'tol': 0.0022743584316052435, 'C': 0.07772127735463577}.
[I 2019-05-20 01:51:27,906] Finished a trial resulted in value: 0.23006625110572052. Current best value is 0.2176318459171829 with parameters: {'tol': 0.0022743584316052435, 'C': 0.07772127735463577}.
[I 2019-05-20 01:55:18,791] Finished a trial resulted in value: 0.21302681117529873. Current best value is 0.21302681117529873 with parameters: {'tol': 0.009936216362271008, 'C': 0.07109160433854167}.
[I 2019-05-20 01:58:30,613] Finished a trial resulted in value: 0.124556669153277. Current best value is 0.124556669153277 with parameters: {'tol': 0.00029965763316254653, 'C': 0.016552321684991808}.
[I 2019-05-20 02:01:42,979] Finished a trial resulted in value: 0.2220427959211385. Current best value is 0.124556669153277 with parameters: {'tol': 0.00029965763316254653, 'C': 0.016552321684991808

[I 2019-05-20 06:04:45,844] Finished a trial resulted in value: 0.17070909697717307. Current best value is 0.0001580228836165994 with parameters: {'tol': 0.006867385429730456, 'C': 0.00015719886906617308}.
[I 2019-05-20 06:07:58,299] Finished a trial resulted in value: 0.16238894193057757. Current best value is 0.0001580228836165994 with parameters: {'tol': 0.006867385429730456, 'C': 0.00015719886906617308}.


In [28]:
for trial_i in mytrial:
    db.insert(trial_i)

In [31]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 1')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
137,2019-05-20 09:01:46.433971,tune 1,15,2.043196,0.003123,2.091750,0.015234,0.048555
170,2019-05-20 10:48:16.389100,tune 1,15,2.044071,0.003133,2.091971,0.015201,0.047900
81,2019-05-20 06:01:33.199607,tune 1,15,2.044651,0.003136,2.092031,0.015176,0.047380
118,2019-05-20 08:00:40.156557,tune 1,15,2.043713,0.003133,2.092043,0.015241,0.048331
31,2019-05-20 03:20:26.879165,tune 1,15,2.044751,0.003129,2.092091,0.015182,0.047340
39,2019-05-20 03:46:04.667985,tune 1,15,2.045696,0.003142,2.092262,0.015109,0.046565
104,2019-05-20 07:15:30.785332,tune 1,15,2.046396,0.003144,2.092399,0.015011,0.046004
157,2019-05-20 10:06:25.296764,tune 1,15,2.046736,0.003153,2.092413,0.015026,0.045678
184,2019-05-20 11:33:21.369361,tune 1,15,2.047225,0.003154,2.092593,0.014990,0.045368
145,2019-05-20 09:27:35.568088,tune 1,15,2.048043,0.003155,2.092812,0.014963,0.044770


In [ ]:
param = copy.deepcopy(df_trial.loc[137]['param'])
param['kfold']['type'] = 'stratified'
# run one try
mytrial=[]
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='remodel 137 use stratified')

In [36]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [36]:
df_trial.tail(1)[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
203,2019-05-20 12:49:47.634780,15,2.054332,0.000002,2.055705,0.000023,0.001373,remodel 137 use stratified


In [37]:
db.commit()

In [12]:
mytrial = []
#  tune hypterparameters
def objective(trial):
        
    tol = trial.suggest_uniform('tol', 0.0001, 0.01)
    C = trial.suggest_uniform('C', 0.0001, 0.1)
        
    args={'algorithm': {'cls': 'SVR',
      'fit': {},
      'init': {'kernel': 'rbf',
       'degree': 2,
       'gamma': 'auto',
       'coef0': 0.0,
       'tol': tol,
       'C': C,
       'epsilon': 0.01,
       'shrinking': True}},
     'columns': common_columns75,

     'kfold': {'n_splits': 3,
      'random_state': 1985,
      'shuffle': True,
      'type': 'stratified'},#stratified
     'scaler': {'cls': 'StandardScaler',
                'init':{}}}
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1 by stratified')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-20 23:13:00,197] Finished a trial resulted in value: 0.005863860497126305. Current best value is 0.005863860497126305 with parameters: {'tol': 0.002229953469187023, 'C': 0.04786123679956754}.
[I 2019-05-20 23:16:14,120] Finished a trial resulted in value: 0.0014903025182072332. Current best value is 0.0014903025182072332 with parameters: {'tol': 0.0067548931833500145, 'C': 0.002057647504840425}.
[I 2019-05-20 23:19:26,133] Finished a trial resulted in value: 0.0047002733413062155. Current best value is 0.0014903025182072332 with parameters: {'tol': 0.0067548931833500145, 'C': 0.002057647504840425}.
[I 2019-05-20 23:22:38,202] Finished a trial resulted in value: 0.00836412703908799. Current best value is 0.0014903025182072332 with parameters: {'tol': 0.0067548931833500145, 'C': 0.002057647504840425}.
[I 2019-05-20 23:25:50,285] Finished a trial resulted in value: 0.006895502023074254. Current best value is 0.0014903025182072332 with parameters: {'tol': 0.0067548931833500145, 

[I 2019-05-21 01:21:35,342] Finished a trial resulted in value: 0.004207069401847464. Current best value is 0.00046830994243634764 with parameters: {'tol': 0.0099151681733301, 'C': 0.000243298481359705}.
[I 2019-05-21 01:24:47,764] Finished a trial resulted in value: 0.008261022743887353. Current best value is 0.00046830994243634764 with parameters: {'tol': 0.0099151681733301, 'C': 0.000243298481359705}.
[I 2019-05-21 01:28:00,411] Finished a trial resulted in value: 0.0021760578900169316. Current best value is 0.00046830994243634764 with parameters: {'tol': 0.0099151681733301, 'C': 0.000243298481359705}.
[I 2019-05-21 01:31:12,668] Finished a trial resulted in value: 0.0064503272307339776. Current best value is 0.00046830994243634764 with parameters: {'tol': 0.0099151681733301, 'C': 0.000243298481359705}.
[I 2019-05-21 01:34:24,756] Finished a trial resulted in value: 0.0031654387308593275. Current best value is 0.00046830994243634764 with parameters: {'tol': 0.0099151681733301, 'C': 

[I 2019-05-21 03:27:11,434] Finished a trial resulted in value: 0.005895593313978574. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 03:30:23,400] Finished a trial resulted in value: 0.0036978760850028525. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 03:33:35,572] Finished a trial resulted in value: 0.004698185729571403. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 03:36:47,548] Finished a trial resulted in value: 0.002869254438447604. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 03:39:59,606] Finished a trial resulted in value: 0.0026903767842579845. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074

[I 2019-05-21 05:32:26,851] Finished a trial resulted in value: 0.004355649805953139. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 05:35:39,247] Finished a trial resulted in value: 0.003693979242503929. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 05:38:51,462] Finished a trial resulted in value: 0.005828699385667012. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 05:42:03,523] Finished a trial resulted in value: 0.004605913849020823. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 05:45:17,564] Finished a trial resulted in value: 0.0014330405685063932. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074,

[I 2019-05-21 07:38:08,493] Finished a trial resulted in value: 0.0025196782284758095. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 07:41:30,890] Finished a trial resulted in value: 0.00019453941041122072. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 07:44:44,671] Finished a trial resulted in value: 0.0015594615883939324. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 07:47:56,648] Finished a trial resulted in value: 0.00283978785685707. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 07:51:19,723] Finished a trial resulted in value: 9.947739803117267e-05. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.005770341777610

[I 2019-05-21 09:45:20,380] Finished a trial resulted in value: 0.008248906167589758. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 09:50:06,247] Finished a trial resulted in value: 0.0013522625032704875. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 09:53:57,764] Finished a trial resulted in value: 0.0003528011843439596. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.
[I 2019-05-21 09:58:08,174] Finished a trial resulted in value: 0.007681609834708156. Current best value is 8.033638978145079e-05 with parameters: {'tol': 0.00577034177761074, 'C': 0.00012538846601435182}.


In [15]:
for trial_i in mytrial:
    db.insert(trial_i)

In [16]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 1 by stratified')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
238,2019-05-21 01:02:20.908098,tune 1 by stratified,15,2.000519,0.000002,2.004971,0.000002,0.004453
438,2019-05-21 01:02:20.908098,tune 1 by stratified,15,2.000519,0.000002,2.004971,0.000002,0.004453
341,2019-05-21 06:33:42.015900,tune 1 by stratified,15,2.001124,0.000002,2.005533,0.000002,0.004409
541,2019-05-21 06:33:42.015900,tune 1 by stratified,15,2.001124,0.000002,2.005533,0.000002,0.004409
450,2019-05-21 01:40:49.358356,tune 1 by stratified,15,2.001838,0.000002,2.006189,0.000002,0.004350
250,2019-05-21 01:40:49.358356,tune 1 by stratified,15,2.001838,0.000002,2.006189,0.000002,0.004350
213,2019-05-20 23:41:50.634568,tune 1 by stratified,15,2.002010,0.000002,2.006333,0.000002,0.004324
413,2019-05-20 23:41:50.634568,tune 1 by stratified,15,2.002010,0.000002,2.006333,0.000002,0.004324
511,2019-05-21 04:57:12.244474,tune 1 by stratified,15,2.002094,0.000002,2.006418,0.000002,0.004324
311,2019-05-21 04:57:12.244474,tune 1 by stratified,15,2.002094,0.000002,2.006418,0.000002,0.004324


In [17]:
param = copy.deepcopy(df_trial.loc[238]['param'])
param['kfold']['type'] = 'group'
# run one try
mytrial=[]
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='remodel 238 use group')

In [18]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [19]:
df_trial.tail(1)[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
604,2019-05-21 10:31:01.450355,15,1.976886,0.002282,2.087706,0.015596,0.110819


In [20]:
db.commit()